# Interacción Agente-a-Agente (A2A): Consultor Técnico Aeroespacial Remoto

## Descripción

En este ejercicio, trabajarás con comunicación Agente-a-Agente (A2A), un patrón donde un agente local se comunica con un agente remoto alojado en un servidor FastAPI. El agente remoto proporciona consultoría técnica en ingeniería aeroespacial.

### Requisitos Previos
- El servidor remoto debe estar ejecutándose: `python remote-agent/agent.py`
- Configuración de Azure AI Foundry completada (PROJECT_ENDPOINT, MODEL_DEPLOYMENT_NAME)
- Protocolo HTTP disponible en localhost:8000

Este notebook demuestra una interacción Agente-a-Agente (A2A) donde un agente local se comunica con un agente remoto ejecutándose en `http://127.0.0.1:8000`. El agente remoto proporciona soporte técnico especializado en ingeniería aeroespacial.

## 1. Importar Librerías

In [ ]:
import httpx
from a2a.client import A2ACardResolver
from agent_framework.a2a import A2AAgent

## 2. Inicializar Comunicación A2A

In [ ]:
async def main():
    # Create an async HTTP client for communication
    async with httpx.AsyncClient(timeout=60.0) as http_client:
        # Initialize the A2A card resolver to retrieve remote agent metadata
        resolver = A2ACardResolver(
            httpx_client=http_client, base_url="http://127.0.0.1:8000"
        )
        try:
            print("Retrieving agent card from remote agent...")
            agent_card = await resolver.get_agent_card(
                relative_card_path="/.well-known/agent.json"
            )
            print(f"   Agent Name: {agent_card.name}")
            print(f"   Description: {agent_card.description}")
            print(f"   URL: http://127.0.0.1:8000/invoke")
            print("-"*80 + "\n")
            
            # Create an A2A agent instance with the retrieved card
            agent = A2AAgent(
                name=agent_card.name,   
                description=agent_card.description,
                agent_card=agent_card,
                url="http://127.0.0.1:8000/invoke",
            )

            # Create a new conversation thread
            thread = agent.get_new_thread()
            
            # Define the question to send to the remote agent
            question = "¿Qué medidas deben tomarse respecto al torque de nivel según la JSSG?"
            print(f"Sending question to remote agent...")
            
            print("Waiting for remote agent response...\n")
            # Send the question and wait for the response
            response = await agent.run(question, thread=thread, http_client=http_client)

            print("RESPONSE FROM REMOTE AGENT:")
            print("="*80)
            for message in response.messages:
                print(message.text)
            print("="*80 + "\n")

        except httpx.ConnectError as e:
            print(f"Connection error: {e}")
            print("   Make sure the remote agent server is running on http://127.0.0.1:8000")
        except Exception as e:
            print(f"An error occurred: {e}")

## 3. Ejecutar el Demo A2A

In [ ]:
await main()